之前用的网络都是单层线性网络模型，效果不理想，现在需要对网络模型 进行优化
![](https://ai-studio-static-online.cdn.bcebos.com/a3cec44cfb614703a12d17b3dacf5d9ef92d5a82db2046c1b5f8aba744bb80de)

In [7]:
import paddle
from paddle.vision.transforms import Normalize

In [8]:
# 数据加载函数方便下面调用
def get_MNIST_dataloader():
    # 定义图像归一化处理方法，这里的CHW指图像格式需为 [C通道数，H图像高度，W图像宽度]
    transform = Normalize(mean=[127.5], std=[127.5], data_format='CHW')
    # 下载数据集并初始化 DataSet
    train_dataset = paddle.vision.datasets.MNIST(mode='train', transform=transform)
    test_dataset = paddle.vision.datasets.MNIST(mode='test', transform=transform)

    # 定义并初始化数据读取器
    train_loader = paddle.io.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=1, drop_last=True)
    test_loader = paddle.io.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=1, drop_last=False)

    return train_loader, test_loader


train_loader,_ = get_MNIST_dataloader()

Cache file C:\Users\shimon\.cache\paddle\dataset\mnist\t10k-images-idx3-ubyte.gz not found, downloading https://dataset.bj.bcebos.com/mnist/t10k-images-idx3-ubyte.gz 
Begin to download


item 403/403 [============================>.] - ETA: 0s - 13ms/item


Download finished
Cache file C:\Users\shimon\.cache\paddle\dataset\mnist\t10k-labels-idx1-ubyte.gz not found, downloading https://dataset.bj.bcebos.com/mnist/t10k-labels-idx1-ubyte.gz 
Begin to download


item 2/2 [===========================>..] - ETA: 0s - 2ms/item


Download finished
d:\miniconda3\envs\paddle_env\lib\site-packages\paddle\fluid\reader.py:486: UserWarning: DataLoader with multi-process mode is not supported on MacOs and Windows currently. Please use signle-process mode with num_workers = 0 instead
  warnings.warn(


![](https://ai-studio-static-online.cdn.bcebos.com/2173259df0704335b230ec158be0427677b9c77fd42348a28f2f8adf1ac1c706)
* 输入层：将数据输入给神经网络。在该任务中，输入层的尺度为28×28的像素值。
* 隐含层：增加网络深度和复杂度，隐含层的节点数是可以调整的，节点数越多，神经网络表示能力越强，参数量也会增加。在该任务中，中间的两个隐含层为10×10的结构，通常隐含层会比输入层的尺寸小，以便对关键信息做抽象，激活函数使用常见的Sigmoid函数。
* 输出层：输出网络计算结果，输出层的节点数是固定的。如果是回归问题，节点数量为需要回归的数字数量。如果是分类问题，则是分类标签的数量。在该任务中，模型的输出是回归一个数字，输出层的尺寸为1。

In [9]:
import paddle
import paddle.nn.functional as F
from paddle.nn import Linear

# 定义多层全连接神经网络
class MNIST(paddle.nn.Layer):
    def __init__(self):
        super(MNIST, self).__init__()
        # 定义两层全连接隐含层，输出维度是10，当前设定隐含节点数为10，可根据任务调整
        self.fc1 = Linear(in_features=784, out_features=10)
        self.fc2 = Linear(in_features=10, out_features=10)
        # 定义一层全连接输出层，输出维度是1
        self.fc3 = Linear(in_features=10, out_features=1)
    
    # 定义网络的前向计算，隐含层激活函数为sigmoid，输出层不使用激活函数
    def forward(self, inputs):
        inputs = paddle.reshape(inputs, [inputs.shape[0], 784])
        outputs1 = self.fc1(inputs)
        outputs1 = F.sigmoid(outputs1)
        outputs2 = self.fc2(outputs1)
        outputs2 = F.sigmoid(outputs2)
        outputs_final = self.fc3(outputs2)
        return outputs_final

In [10]:
model = MNIST()
params_info = paddle.summary(model, (1, 1, 28, 28))
print(params_info)

---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Linear-7          [[1, 784]]            [1, 10]             7,850     
   Linear-8          [[1, 10]]             [1, 10]              110      
   Linear-9          [[1, 10]]              [1, 1]              11       
Total params: 7,971
Trainable params: 7,971
Non-trainable params: 0
---------------------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.03
Estimated Total Size (MB): 0.03
---------------------------------------------------------------------------

{'total_params': 7971, 'trainable_params': 7971}


In [11]:
#网络结构部分之后的代码，保持不变
def train(model):
    model.train()
    
    # 使用SGD优化器，learning_rate设置为0.01
    opt = paddle.optimizer.SGD(learning_rate=0.01, parameters=model.parameters())
    # 训练5轮
    EPOCH_NUM = 10

    loss_list = []
    
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            #准备数据
            images, labels = data

            images = paddle.to_tensor(images)

            labels = paddle.to_tensor(labels, dtype="float32")
            
            #前向计算的过程
            predicts = model(images)
            
            #计算损失，取一个批次样本损失的平均值
            loss = F.square_error_cost(predicts, labels)
            avg_loss = paddle.mean(loss)

            #每训练200批次的数据，打印下当前Loss的情况
            if batch_id % 200 == 0:
                loss_list.append(avg_loss.numpy()[0])
                print("epoch: {}, batch: {}, loss is: {}".format(epoch_id, batch_id, avg_loss.numpy()))
            
            #后向传播，更新参数的过程
            avg_loss.backward()
            # 最小化loss,更新参数
            opt.step()
            # 清除梯度
            opt.clear_grad()

    #保存模型参数
    paddle.save(model.state_dict(), 'mnist.pdparams')
    return loss_list

model = MNIST()
loss_list = train(model)

epoch: 0, batch: 0, loss is: [23.48772]
epoch: 0, batch: 200, loss is: [4.5388184]
epoch: 0, batch: 400, loss is: [3.2588058]
epoch: 0, batch: 600, loss is: [3.3792841]
epoch: 0, batch: 800, loss is: [2.800582]
epoch: 1, batch: 0, loss is: [2.0283704]
epoch: 1, batch: 200, loss is: [2.419256]
epoch: 1, batch: 400, loss is: [0.88706195]
epoch: 1, batch: 600, loss is: [3.0536935]
epoch: 1, batch: 800, loss is: [3.0816107]
epoch: 2, batch: 0, loss is: [1.6371665]
epoch: 2, batch: 200, loss is: [1.0735066]
epoch: 2, batch: 400, loss is: [1.1204636]
epoch: 2, batch: 600, loss is: [2.992417]
epoch: 2, batch: 800, loss is: [2.2066522]
epoch: 3, batch: 0, loss is: [1.3851948]
epoch: 3, batch: 200, loss is: [1.1568284]
epoch: 3, batch: 400, loss is: [1.6903161]
epoch: 3, batch: 600, loss is: [0.8489225]
epoch: 3, batch: 800, loss is: [1.0682223]
epoch: 4, batch: 0, loss is: [0.76268595]
epoch: 4, batch: 200, loss is: [1.4886348]
epoch: 4, batch: 400, loss is: [1.2604399]
epoch: 4, batch: 600, l

In [12]:
# 定义 SimpleNet 网络结构
import paddle
from paddle.nn import Conv2D, MaxPool2D, Linear
import paddle.nn.functional as F
# 多层卷积神经网络实现
class MNIST(paddle.nn.Layer):
     def __init__(self):
         super(MNIST, self).__init__()
         
         # 定义卷积层，输出特征通道out_channels设置为20，卷积核的大小kernel_size为5，卷积步长stride=1，padding=2
         self.conv1 = Conv2D(in_channels=1, out_channels=20, kernel_size=5, stride=1, padding=2)
         # 定义池化层，池化核的大小kernel_size为2，池化步长为2
         self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
         # 定义卷积层，输出特征通道out_channels设置为20，卷积核的大小kernel_size为5，卷积步长stride=1，padding=2
         self.conv2 = Conv2D(in_channels=20, out_channels=20, kernel_size=5, stride=1, padding=2)
         # 定义池化层，池化核的大小kernel_size为2，池化步长为2
         self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
         # 定义一层全连接层，输出维度是1
         self.fc = Linear(in_features=980, out_features=1)
         
    # 定义网络前向计算过程，卷积后紧接着使用池化层，最后使用全连接层计算最终输出
    # 卷积层激活函数使用Relu，全连接层不使用激活函数
     def forward(self, inputs):
         x = self.conv1(inputs)
         x = F.relu(x)
         x = self.max_pool1(x)
         x = self.conv2(x)
         x = F.relu(x)
         x = self.max_pool2(x)
         x = paddle.reshape(x, [x.shape[0], -1])
         x = self.fc(x)
         return x

In [13]:
model = MNIST()
params_info = paddle.summary(model, (1, 1, 28, 28))
print(params_info)

---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv2D-1       [[1, 1, 28, 28]]     [1, 20, 28, 28]          520      
  MaxPool2D-1    [[1, 20, 28, 28]]     [1, 20, 14, 14]           0       
   Conv2D-2      [[1, 20, 14, 14]]     [1, 20, 14, 14]        10,020     
  MaxPool2D-2    [[1, 20, 14, 14]]      [1, 20, 7, 7]            0       
   Linear-13         [[1, 980]]             [1, 1]              981      
Total params: 11,521
Trainable params: 11,521
Non-trainable params: 0
---------------------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.19
Params size (MB): 0.04
Estimated Total Size (MB): 0.23
---------------------------------------------------------------------------

{'total_params': 11521, 'trainable_params': 11521}


In [14]:
#网络结构部分之后的代码，保持不变
def train(model):
    model.train()
    
    learning_rate = 0.001 

    # 使用SGD优化器，设置learning_rate
    opt = paddle.optimizer.SGD(learning_rate=learning_rate, parameters=model.parameters())
    # 训练5轮
    EPOCH_NUM = 10
    # MNIST图像高和宽
    IMG_ROWS, IMG_COLS = 28, 28

    loss_list = []

    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            #准备数据
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.to_tensor(labels, dtype="float32")
            
            #前向计算的过程
            predicts = model(images) # [batch_size, 1]

            #计算损失，取一个批次样本损失的平均值
            loss = F.square_error_cost(predicts, labels)
            avg_loss = paddle.mean(loss)

            #每训练200批次的数据，打印下当前Loss的情况
            if batch_id % 200 == 0:
                loss_list.append(avg_loss.numpy()[0])
                print("epoch: {}, batch: {}, loss is: {}".format(epoch_id, batch_id, avg_loss.numpy()))
            
            #后向传播，更新参数的过程
            avg_loss.backward()
            # 最小化loss,更新参数
            opt.step()
            # 清除梯度
            opt.clear_grad()

    #保存模型参数
    paddle.save(model.state_dict(), './model/mnist.pdparams')

    return loss_list

model = MNIST()
loss_list_conv = train(model)

epoch: 0, batch: 0, loss is: [36.399822]
epoch: 0, batch: 200, loss is: [2.3570137]
epoch: 0, batch: 400, loss is: [2.2580333]
epoch: 0, batch: 600, loss is: [2.4587507]
epoch: 0, batch: 800, loss is: [1.557093]
epoch: 1, batch: 0, loss is: [2.043179]
epoch: 1, batch: 200, loss is: [1.6813451]
epoch: 1, batch: 400, loss is: [1.181281]
epoch: 1, batch: 600, loss is: [2.4105873]
epoch: 1, batch: 800, loss is: [2.0873356]
epoch: 2, batch: 0, loss is: [1.5336311]
epoch: 2, batch: 200, loss is: [2.0856245]
epoch: 2, batch: 400, loss is: [0.99403465]
epoch: 2, batch: 600, loss is: [1.4321723]
epoch: 2, batch: 800, loss is: [1.2517223]
epoch: 3, batch: 0, loss is: [1.3699272]
epoch: 3, batch: 200, loss is: [0.92330563]
epoch: 3, batch: 400, loss is: [0.89157975]
epoch: 3, batch: 600, loss is: [0.7458833]
epoch: 3, batch: 800, loss is: [1.1637061]


In [ ]:
import matplotlib.pyplot as plt
def plot_two_losses(loss_list_1, loss_list_2):
    plt.figure(figsize=(10,5))
    
    freqs = [i for i in range(len(loss_list_1))]

    # 绘制训练损失变化曲线
    plt.plot(freqs, loss_list_1, color='#e4007f', label="Train loss1")
    plt.plot(freqs, loss_list_2, color='#f19ec2', linestyle='--', label="Train loss2")
    
    # 绘制坐标轴和图例
    plt.ylabel("loss", fontsize='large')
    plt.xlabel("freq", fontsize='large')
    plt.legend(loc='upper right', fontsize='x-large')
    
    plt.show()

plot_two_losses(loss_list, loss_list_conv)